In [ ]:
GIS_DIR = "/content/Data_GIS_New/Data GIS"  

# find the PM2.5 shapefile
pm_shp = None
for p in glob.glob(os.path.join(GIS_DIR, "*.shp")):
    if os.path.basename(p).lower().startswith("pm2.5"):
        pm_shp = p
        break
assert pm_shp is not None, "PM2.5.shp not found in GIS_DIR."

# read shapefile (pyshp)
r = shapefile.Reader(pm_shp)
fields = [f[0] for f in r.fields if f[0] != "DeletionFlag"]
recs = r.records()
shps = r.shapes()

# build dataframe with attributes + coords (x,y)
rows = []
for rec, shp in zip(recs, shps):
    attrs = dict(zip(fields, rec))
    # take first point (assuming point shapefile)
    x, y = shp.points[0]
    attrs.update({"x": x, "y": y})
    rows.append(attrs)

pm_df = pd.DataFrame(rows)
print("PM columns:", pm_df.columns.tolist())
print(pm_df.head())


PM columns: ['OBJECTID', 'station', 'pm_2_5', 'Latitude', 'Longitude', 'x', 'y']
   OBJECTID   station  pm_2_5      Latitude    Longitude            x  \
0         2  aghsasie    24.0  3.961416e+06  543746.7260  543746.7260   
1         3  park roz    27.0  3.955132e+06  524222.9926  524222.9926   
2         4    poonak    21.0  3.957635e+06  529983.0336  529983.0336   
3         5    pirozi    34.0  3.950342e+06  544671.7644  544671.7644   
4         6   tarbiat    33.0  3.952686e+06  534904.7643  534904.7643   

              y  
0  3.961416e+06  
1  3.955132e+06  
2  3.957635e+06  
3  3.950342e+06  
4  3.952686e+06  


In [6]:
# read shapefile CRS from .prj (if present)
prj_path = pm_shp.replace(".shp", ".prj")
crs_vec = CRS.from_wkt(open(prj_path).read()) if os.path.exists(prj_path) else None
print("Vector CRS:", crs_vec)

def make_transformer(crs_from, crs_to):
    if (crs_from is None) or (crs_to is None):
        return None
    if CRS.from_user_input(crs_from) == CRS.from_user_input(crs_to):
        return None
    return Transformer.from_crs(crs_from, crs_to, always_xy=True)


Vector CRS: PROJCS["WGS_1984_UTM_Zone_39N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["false_easting",500000.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",51.0],PARAMETER["scale_factor",0.9996],PARAMETER["latitude_of_origin",0.0],UNIT["Meter",1.0]]


In [7]:
# collect raster files (ignore .ovr)
rasters = {}
for path in glob.glob(os.path.join(GIS_DIR, "*.tif")):
    name = os.path.splitext(os.path.basename(path))[0]
    if name.endswith(".tif"): name = name[:-4]
    rasters[name] = path

print("Rasters found:", list(rasters.keys()))

def sample_raster_column(df, xcol, ycol, ras_path, out_col):
    with rasterio.open(ras_path) as ds:
        tx = make_transformer(crs_vec, ds.crs)
        if tx is not None:
            pts = [tx.transform(x, y) for x, y in zip(df[xcol].values, df[ycol].values)]
        else:
            pts = list(zip(df[xcol].values, df[ycol].values))
        # ds.sample expects [(x,y), ...] in raster CRS
        vals = list(ds.sample(pts))
        vals = [float(v[0]) if (v is not None and len(v)) else np.nan for v in vals]
    df[out_col] = vals
    return df

# Map friendly names (adjust if you want different labels)
name_map = {
    "NDVI": "ndvi",
    "Temperature": "temp",
    "Humidity": "humidity",
    "Rainfall": "rain",
    "Wind speed": "wind",
    "Population density": "pop_density",
}
for key, col in name_map.items():
    # find raster key that matches (case-insensitive, space-insensitive)
    cand = next((k for k in rasters if k.lower().replace(" ","") == key.lower().replace(" ","")), None)
    if cand:
        pm_df = sample_raster_column(pm_df, "x", "y", rasters[cand], col)
        print(f"Sampled: {key} -> {col}")
    else:
        print(f"Missing raster for: {key}")

print(pm_df.head())


Rasters found: ['Population density', 'Wind speed', 'Rainfall', 'NDVI', 'Temperature', 'Humidity', 'Distance to street', 'Distance to industrial', 'Elevation']
Sampled: NDVI -> ndvi
Sampled: Temperature -> temp
Sampled: Humidity -> humidity
Sampled: Rainfall -> rain
Sampled: Wind speed -> wind
Sampled: Population density -> pop_density
   OBJECTID   station  pm_2_5      Latitude    Longitude            x  \
0         2  aghsasie    24.0  3.961416e+06  543746.7260  543746.7260   
1         3  park roz    27.0  3.955132e+06  524222.9926  524222.9926   
2         4    poonak    21.0  3.957635e+06  529983.0336  529983.0336   
3         5    pirozi    34.0  3.950342e+06  544671.7644  544671.7644   
4         6   tarbiat    33.0  3.952686e+06  534904.7643  534904.7643   

              y      ndvi       temp   humidity        rain      wind  \
0  3.961416e+06  0.186091  16.381969  39.679241  325.161865  2.164852   
1  3.955132e+06  0.231287  17.617378  36.651649  251.852448  2.541904   
2  3

In [8]:
# try to guess PM column name; change "PM_value" below if needed
# Look through columns for a PM-ish field:
pm_candidates = [c for c in pm_df.columns if c.lower().startswith("pm")]
print("PM-candidate columns:", pm_candidates)

OUT_CSV = "/content/station_static_features.csv"
pm_df.to_csv(OUT_CSV, index=False)
print("Saved:", OUT_CSV)


PM-candidate columns: ['pm_2_5']
Saved: /content/station_static_features.csv


In [9]:
from scipy.signal import savgol_filter
from scipy.interpolate import UnivariateSpline
import numpy as np
import pandas as pd

# Load back the dataset we just saved
df = pd.read_csv("/content/station_static_features.csv")

# --- Savitzky–Golay smoothing on PM2.5 ---
df["pm_2_5_smooth"] = savgol_filter(df["pm_2_5"], window_length=7, polyorder=2, mode="nearest")

# --- Example: spline interpolation for missing values in predictors ---
def spline_fill(series):
    x = np.arange(len(series))
    mask = ~np.isnan(series)
    if mask.sum() < 4:   # not enough points for spline
        return series.fillna(method="ffill").fillna(method="bfill")
    spline = UnivariateSpline(x[mask], series[mask], k=3, s=0)
    return spline(x)

for col in ["temp", "humidity", "rain", "wind", "ndvi"]:
    if df[col].isna().any():
        df[col] = spline_fill(df[col].values)

print(df.head())


   OBJECTID   station  pm_2_5      Latitude    Longitude            x  \
0         2  aghsasie    24.0  3.961416e+06  543746.7260  543746.7260   
1         3  park roz    27.0  3.955132e+06  524222.9926  524222.9926   
2         4    poonak    21.0  3.957635e+06  529983.0336  529983.0336   
3         5    pirozi    34.0  3.950342e+06  544671.7644  544671.7644   
4         6   tarbiat    33.0  3.952686e+06  534904.7643  534904.7643   

              y      ndvi       temp   humidity        rain      wind  \
0  3.961416e+06  0.186091  16.381969  39.679241  325.161865  2.164852   
1  3.955132e+06  0.231287  17.617378  36.651649  251.852448  2.541904   
2  3.957635e+06  0.171851  17.537899  36.835045  273.974121  2.214953   
3  3.950342e+06  0.070099  15.905035  41.332394  394.644257  1.235514   
4  3.952686e+06  0.049471  17.555529  36.868725  284.113342  2.023376   

   pop_density  pm_2_5_smooth  
0    48.801765      23.476190  
1    25.537809      24.714286  
2     2.087641      26.190

In [10]:
import numpy as np

# IDW interpolation function
def idw_interpolation(xy_known, values, xy_target, power=2):
    """Inverse Distance Weighting (IDW) interpolation."""
    dists = np.linalg.norm(xy_known - xy_target, axis=1)
    weights = 1 / (dists**power + 1e-8)  # avoid division by zero
    return np.sum(weights * values) / np.sum(weights)

# Example: suppose you have 5 meteorological stations dataframe
# (replace with your actual met data table)
met_stations = pd.DataFrame({
    "Longitude": [51.3, 51.4, 51.5, 51.6, 51.7],
    "Latitude": [35.7, 35.8, 35.9, 36.0, 36.1],
    "temp": [17.0, 16.5, 15.8, 16.2, 17.3],
    "humidity": [40, 38, 37, 41, 39],
    "wind": [2.2, 2.5, 2.0, 2.3, 2.1],
    "rain": [300, 280, 310, 295, 305]
})

# Convert to numpy coords
xy_known = met_stations[["Longitude", "Latitude"]].values

# Apply IDW to each AQ station
for var in ["temp", "humidity", "wind", "rain"]:
    interpolated = []
    for i, row in df.iterrows():
        xy_target = np.array([row["Longitude"], row["Latitude"]])
        value = idw_interpolation(xy_known, met_stations[var].values, xy_target)
        interpolated.append(value)
    df[var + "_idw"] = interpolated

print(df.head())


   OBJECTID   station  pm_2_5      Latitude    Longitude            x  \
0         2  aghsasie    24.0  3.961416e+06  543746.7260  543746.7260   
1         3  park roz    27.0  3.955132e+06  524222.9926  524222.9926   
2         4    poonak    21.0  3.957635e+06  529983.0336  529983.0336   
3         5    pirozi    34.0  3.950342e+06  544671.7644  544671.7644   
4         6   tarbiat    33.0  3.952686e+06  534904.7643  534904.7643   

              y      ndvi       temp   humidity        rain      wind  \
0  3.961416e+06  0.186091  16.381969  39.679241  325.161865  2.164852   
1  3.955132e+06  0.231287  17.617378  36.651649  251.852448  2.541904   
2  3.957635e+06  0.171851  17.537899  36.835045  273.974121  2.214953   
3  3.950342e+06  0.070099  15.905035  41.332394  394.644257  1.235514   
4  3.952686e+06  0.049471  17.555529  36.868725  284.113342  2.023376   

   pop_density  pm_2_5_smooth  temp_idw  humidity_idw  wind_idw  rain_idw  
0    48.801765      23.476190     16.56       

In [11]:
# ==== Temporal + Spatial split ====
import numpy as np

df_split = df.copy()

# Prefer IDW meteorological features
feature_cols = ["temp_idw", "humidity_idw", "wind_idw", "rain_idw",
                "ndvi", "pop_density"]

# Use smoothed PM2.5 as target
target_col = "pm_2_5_smooth"

# If you already have a "date" column, activate year-based split
if "date" in df_split.columns:
    df_split["date"] = pd.to_datetime(df_split["date"])
    df_split["year"] = df_split["date"].dt.year
else:
    # Dummy year column (all NaN) → only spatial split will apply
    df_split["year"] = np.nan

# Temporal split (2014–2015 train, 2016 test)
temporal_train_mask = df_split["year"].between(2014, 2015) if df_split["year"].notna().any() else pd.Series(True, index=df_split.index)
temporal_test_mask  = df_split["year"] == 2016              if df_split["year"].notna().any() else pd.Series(True, index=df_split.index)

# Spatial split (hold out 20% of stations)
rng = np.random.RandomState(42)
stations = df_split["station"].astype(str).unique()
n_holdout = max(1, int(0.2 * len(stations)))
holdout_stations = rng.choice(stations, size=n_holdout, replace=False).tolist()
print("Spatial hold-out stations:", holdout_stations)

spatial_holdout_mask = df_split["station"].astype(str).isin(holdout_stations)

# Train = temporal train years & not in spatial holdout
train_mask = temporal_train_mask & (~spatial_holdout_mask)
# Test = temporal test year OR spatial holdout stations
test_mask = temporal_test_mask | spatial_holdout_mask

train_df = df_split.loc[train_mask].reset_index(drop=True)
test_df  = df_split.loc[test_mask].reset_index(drop=True)

print(f"Train rows: {len(train_df)}, Test rows: {len(test_df)}")

X_train = train_df[feature_cols].values
y_train = train_df[target_col].values
X_test  = test_df[feature_cols].values
y_test  = test_df[target_col].values

print("Feature matrix shape:", X_train.shape, "Target shape:", y_train.shape)


Spatial hold-out stations: ['aghsasie', 'shahrdari11', 'sharif', 'park roz']
Train rows: 18, Test rows: 22
Feature matrix shape: (18, 6) Target shape: (18,)


In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

def evaluate(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2   = r2_score(y_true, y_pred)
    sd   = np.std(y_true - y_pred)
    return {"RMSE": float(rmse), "R2": float(r2), "SD": float(sd)}

print("Scaled shapes:", X_train_s.shape, X_test_s.shape)


Scaled shapes: (18, 6) (22, 6)


In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# ---- Random Forest baseline ----
rf = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
rf.fit(X_train_s, y_train)
pred_rf = rf.predict(X_test_s)
print("Random Forest:", evaluate(y_test, pred_rf))

# ---- SVM baseline ----
svm = SVR(kernel="rbf", C=10.0, gamma="scale")
svm.fit(X_train_s, y_train)
pred_svm = svm.predict(X_test_s)
print("SVM:", evaluate(y_test, pred_svm))


Random Forest: {'RMSE': 3.2659205414723176, 'R2': 0.49848535508258907, 'SD': 3.2396363061433413}
SVM: {'RMSE': 3.9655513932073716, 'R2': 0.26059981272135535, 'SD': 3.965551392991338}


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# ---- DNN baseline ----
dnn = Sequential([
    Dense(64, activation="relu", input_shape=(X_train_s.shape[1],)),
    Dense(32, activation="relu"),
    Dense(1)
])
dnn.compile(optimizer="adam", loss="mse")

dnn.fit(X_train_s, y_train, epochs=50, batch_size=8, verbose=0,
        validation_data=(X_test_s, y_test))

pred_dnn = dnn.predict(X_test_s).ravel()
print("DNN:", evaluate(y_test, pred_dnn))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
DNN: {'RMSE': 20.46603823459775, 'R2': -18.694272973079926, 'SD': 8.417741013413849}


In [15]:
from tensorflow.keras.layers import SimpleRNN, LSTM, Reshape

# ---- Reshape for RNN/LSTM ----
X_train_seq = X_train_s.reshape((X_train_s.shape[0], 1, X_train_s.shape[1]))
X_test_seq  = X_test_s.reshape((X_test_s.shape[0], 1, X_test_s.shape[1]))

# ---- Simple RNN baseline ----
rnn = Sequential([
    SimpleRNN(32, input_shape=(1, X_train_s.shape[1])),
    Dense(1)
])
rnn.compile(optimizer="adam", loss="mse")

rnn.fit(X_train_seq, y_train, epochs=50, batch_size=8, verbose=0,
        validation_data=(X_test_seq, y_test))

pred_rnn = rnn.predict(X_test_seq).ravel()
print("RNN:", evaluate(y_test, pred_rnn))

# ---- LSTM baseline ----
lstm = Sequential([
    LSTM(32, input_shape=(1, X_train_s.shape[1])),
    Dense(1)
])
lstm.compile(optimizer="adam", loss="mse")

lstm.fit(X_train_seq, y_train, epochs=50, batch_size=8, verbose=0,
         validation_data=(X_test_seq, y_test))

pred_lstm = lstm.predict(X_test_seq).ravel()
print("LSTM:", evaluate(y_test, pred_lstm))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
RNN: {'RMSE': 30.957073531459006, 'R2': -44.06010385772256, 'SD': 4.561611884384506}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
LSTM: {'RMSE': 31.287072561060288, 'R2': -45.025895776317135, 'SD': 4.721313452882025}


In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.callbacks import EarlyStopping

tf.random.set_seed(42)
np.random.seed(42)

# DNN (with proper Input layer)
dnn = Sequential([
    Input(shape=(X_train_s.shape[1],)),
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
    Dense(1)
])
dnn.compile(optimizer="adam", loss="mse")
dnn.fit(
    X_train_s, y_train,
    validation_split=0.25,
    epochs=200, batch_size=4, verbose=0,
    callbacks=[EarlyStopping(patience=20, restore_best_weights=True)]
)
pred_dnn = dnn.predict(X_test_s).ravel()
print("DNN (fixed):", evaluate(y_test, pred_dnn))

# Prep simple seq shaped inputs (seq_len=1 to match tabular)
X_train_seq = X_train_s.reshape((X_train_s.shape[0], 1, X_train_s.shape[1]))
X_test_seq  = X_test_s.reshape((X_test_s.shape[0], 1, X_test_s.shape[1]))

# RNN
rnn = Sequential([
    Input(shape=(1, X_train_s.shape[1])),
    SimpleRNN(16),
    Dense(1)
])
rnn.compile(optimizer="adam", loss="mse")
rnn.fit(
    X_train_seq, y_train,
    validation_split=0.25,
    epochs=200, batch_size=4, verbose=0,
    callbacks=[EarlyStopping(patience=20, restore_best_weights=True)]
)
pred_rnn = rnn.predict(X_test_seq).ravel()
print("RNN:", evaluate(y_test, pred_rnn))

# LSTM
lstm = Sequential([
    Input(shape=(1, X_train_s.shape[1])),
    LSTM(16),
    Dense(1)
])
lstm.compile(optimizer="adam", loss="mse")
lstm.fit(
    X_train_seq, y_train,
    validation_split=0.25,
    epochs=200, batch_size=4, verbose=0,
    callbacks=[EarlyStopping(patience=20, restore_best_weights=True)]
)
pred_lstm = lstm.predict(X_test_seq).ravel()
print("LSTM:", evaluate(y_test, pred_lstm))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
DNN (fixed): {'RMSE': 14.708908980632525, 'R2': -9.172631485066228, 'SD': 11.682091407398493}


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
RNN: {'RMSE': 21.494907546797634, 'R2': -20.724188068356536, 'SD': 4.46384020702864}


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
LSTM: {'RMSE': 24.888530253678866, 'R2': -28.125332544411624, 'SD': 4.343886208501631}


In [17]:
import copy, random, time
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tensorflow.keras.optimizers import Adam

random.seed(42)

# Search space (keep small given tiny data)
PARAM_SPACE = {
    "units":      [8, 16, 24, 32],
    "dropout":    [0.0, 0.1, 0.2, 0.3],
    "lr":         [1e-3, 5e-4, 1e-4],
    "batch_size": [4, 8, 16],
    "epochs":     [80, 120, 180]
}

def build_lstm(input_shape, units=16, dropout=0.1, lr=1e-3):
    m = tf.keras.Sequential([
        Input(shape=input_shape),
        LSTM(units),
        Dense(1)
    ])
    m.compile(optimizer=Adam(learning_rate=lr), loss="mse")
    return m

def cv_rmse_lstm(params, X_seq, y, n_splits=3):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    rmses = []
    for tr, va in kf.split(X_seq):
        X_tr, X_va = X_seq[tr], X_seq[va]
        y_tr, y_va = y[tr], y[va]

        model = build_lstm(
            input_shape=(X_seq.shape[1], X_seq.shape[2]),
            units=params["units"], dropout=params["dropout"], lr=params["lr"]
        )
        es = EarlyStopping(patience=15, restore_best_weights=True)
        model.fit(
            X_tr, y_tr,
            validation_data=(X_va, y_va),
            epochs=params["epochs"],
            batch_size=params["batch_size"],
            verbose=0,
            callbacks=[es]
        )
        pred = model.predict(X_va, verbose=0).ravel()
        rmses.append(np.sqrt(mean_squared_error(y_va, pred)))
    return float(np.mean(rmses))

def random_params():
    return {k: random.choice(v) for k, v in PARAM_SPACE.items()}

def mutate(params, strength=1):
    newp = copy.deepcopy(params)
    for _ in range(strength):
        key = random.choice(list(PARAM_SPACE.keys()))
        newp[key] = random.choice(PARAM_SPACE[key])
    return newp

def orchard_optimize(X_train_s, y_train, pop=10, gens=6, elite=3):
    # reshape to (N, 1, F)
    X_seq = X_train_s.reshape((X_train_s.shape[0], 1, X_train_s.shape[1]))
    population = [random_params() for _ in range(pop)]
    history = []
    best = None

    for g in range(gens):
        scored = []
        for p in population:
            score = cv_rmse_lstm(p, X_seq, y_train, n_splits=3)
            scored.append((score, p))
        scored.sort(key=lambda x: x[0])
        best = scored[0] if (best is None or scored[0][0] < best[0]) else best
        history.append(best[0])

        # Elitism (keep top-k)
        elites = [copy.deepcopy(p) for _, p in scored[:elite]]

        # Growth + Grafting: mutate elites to explore neighbors
        mutants = [mutate(p, strength=1) for p in elites] + [mutate(p, strength=2) for p in elites]

        # Screening + Pruning: add a few random fresh shoots (diversity) and trim worst
        fresh = [random_params() for _ in range(max(0, pop - len(elites) - len(mutants)))]

        population = elites + mutants + fresh

        print(f"Gen {g+1}/{gens} — best CV RMSE: {best[0]:.4f} | best params: {best[1]}")
    return best, history

start = time.time()
(best_tuple, cv_curve) = orchard_optimize(X_train_s, y_train, pop=10, gens=6, elite=3)
print(f"OA done in {time.time()-start:.1f}s")
best_cv_rmse, best_params = best_tuple[0], best_tuple[1]
best_params, best_cv_rmse


Gen 1/6 — best CV RMSE: 29.9684 | best params: {'units': 16, 'dropout': 0.3, 'lr': 0.001, 'batch_size': 8, 'epochs': 180}
Gen 2/6 — best CV RMSE: 29.2229 | best params: {'units': 24, 'dropout': 0.3, 'lr': 0.001, 'batch_size': 8, 'epochs': 180}
Gen 3/6 — best CV RMSE: 26.3129 | best params: {'units': 32, 'dropout': 0.3, 'lr': 0.001, 'batch_size': 4, 'epochs': 180}
Gen 4/6 — best CV RMSE: 26.3129 | best params: {'units': 32, 'dropout': 0.3, 'lr': 0.001, 'batch_size': 4, 'epochs': 180}
Gen 5/6 — best CV RMSE: 26.3129 | best params: {'units': 32, 'dropout': 0.3, 'lr': 0.001, 'batch_size': 4, 'epochs': 180}
Gen 6/6 — best CV RMSE: 26.2835 | best params: {'units': 32, 'dropout': 0.3, 'lr': 0.001, 'batch_size': 4, 'epochs': 180}
OA done in 3478.7s


({'units': 32, 'dropout': 0.3, 'lr': 0.001, 'batch_size': 4, 'epochs': 180},
 26.283480190963733)

In [18]:
import numpy as np
import copy, random
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

random.seed(42)
np.random.seed(42)

def boa_feature_selection(
    X_train, y_train,              # unscaled train features/target
    feature_names,                 # list[str]
    generations=8, pop_size=12, elite=3,
    phi=0.89                       # penalty weight (paper)
):
    n_feat = X_train.shape[1]
    assert n_feat == len(feature_names)

    # 1) Importance from RF on TRAIN ONLY (normalize to sum=1 for penalty)
    rf_all = RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1)
    rf_all.fit(X_train, y_train)
    imp = rf_all.feature_importances_
    imp = imp / (imp.sum() + 1e-12)

    # Helper: CV RMSE for a given feature mask using RF (fast proxy)
    def cv_rmse(mask, n_splits=3):
        idx = np.where(mask == 1)[0]
        if len(idx) == 0:
            return np.inf
        X = X_train[:, idx]
        kf = KFold(n_splits=min(n_splits, len(y_train)), shuffle=True, random_state=42)
        rmses = []
        for tr, va in kf.split(X):
            X_tr, X_va = X[tr], X[va]
            y_tr, y_va = y_train[tr], y_train[va]
            m = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
            m.fit(X_tr, y_tr)
            p = m.predict(X_va)
            rmses.append(np.sqrt(mean_squared_error(y_va, p)))
        return float(np.mean(rmses)) if len(rmses) > 0 else np.inf

    # Objective = CV_RMSE + log(1 + #features) + phi * penalty(dropping important feats)
    def objective(mask):
        rmse = cv_rmse(mask)
        k = mask.sum()
        # penalty for excluding important features
        penalty = phi * float(np.sum((1 - mask) * imp))
        return rmse + np.log1p(k) + penalty

    # Init population (avoid all-zero)
    population = []
    for _ in range(pop_size):
        m = np.random.randint(0, 2, n_feat)
        if m.sum() == 0:
            m[np.random.randint(0, n_feat)] = 1
        population.append(m)

    best_score, best_mask = np.inf, None
    history = []

    for g in range(generations):
        scored = []
        for m in population:
            s = objective(m)
            scored.append((s, m))
            if s < best_score:
                best_score, best_mask = s, m.copy()

        scored.sort(key=lambda x: x[0])
        elites = [scored[i][1].copy() for i in range(min(elite, len(scored)))]

        # Mutations (growth/grafting)
        mutants = []
        for e in elites:
            # single flip
            m1 = e.copy()
            i = np.random.randint(0, n_feat)
            m1[i] = 1 - m1[i]
            if m1.sum() == 0: m1[i] = 1
            mutants.append(m1)
            # double flip
            m2 = e.copy()
            i, j = np.random.randint(0, n_feat, size=2)
            m2[i] = 1 - m2[i]
            m2[j] = 1 - m2[j]
            if m2.sum() == 0: m2[np.random.randint(0, n_feat)] = 1
            mutants.append(m2)

        # Fresh random individuals (screening/diversity)
        fresh = []
        while len(elites) + len(mutants) + len(fresh) < pop_size:
            m = np.random.randint(0, 2, n_feat)
            if m.sum() == 0:
                m[np.random.randint(0, n_feat)] = 1
            fresh.append(m)

        population = elites + mutants + fresh
        history.append(best_score)
        print(f"BOA gen {g+1}/{generations} — best score: {best_score:.4f} | selected: {best_mask.sum()}")

    sel_idx = np.where(best_mask == 1)[0]
    sel_names = [feature_names[i] for i in sel_idx]
    return best_mask, sel_idx, sel_names, history


In [19]:
# Use TRAIN ONLY to decide features
feature_cols_current = feature_cols[:]  # from earlier split cell
mask, sel_idx, sel_names, boa_hist = boa_feature_selection(
    X_train=X_train, y_train=y_train,
    feature_names=feature_cols_current,
    generations=8, pop_size=12, elite=3, phi=0.89
)

print("Selected features:", sel_names)

# Build reduced matrices
X_train_red = X_train[:, sel_idx]
X_test_red  = X_test[:, sel_idx]

# Re-scale (fit on train only)
from sklearn.preprocessing import StandardScaler
scaler_red = StandardScaler()
X_train_red_s = scaler_red.fit_transform(X_train_red)
X_test_red_s  = scaler_red.transform(X_test_red)

print("Reduced shapes:", X_train_red_s.shape, X_test_red_s.shape)


BOA gen 1/8 — best score: 6.0388 | selected: 1
BOA gen 2/8 — best score: 6.0388 | selected: 1
BOA gen 3/8 — best score: 6.0388 | selected: 1
BOA gen 4/8 — best score: 6.0388 | selected: 1
BOA gen 5/8 — best score: 6.0388 | selected: 1
BOA gen 6/8 — best score: 6.0388 | selected: 1
BOA gen 7/8 — best score: 6.0388 | selected: 1
BOA gen 8/8 — best score: 6.0388 | selected: 1
Selected features: ['rain_idw']
Reduced shapes: (18, 1) (22, 1)


In [20]:
# Reuse orchard_optimize and build_lstm from earlier
# Prepare seq tensors with reduced features
X_train_red_seq = X_train_red_s.reshape((X_train_red_s.shape[0], 1, X_train_red_s.shape[1]))
X_test_red_seq  = X_test_red_s.reshape((X_test_red_s.shape[0], 1, X_test_red_s.shape[1]))

# Slightly fewer generations since feature space is already pruned
(best_tuple_red, cv_curve_red) = orchard_optimize(X_train_red_s, y_train, pop=10, gens=5, elite=3)
best_cv_rmse_red, best_params_red = best_tuple_red[0], best_tuple_red[1]
print("Best OA-LSTM (reduced) CV RMSE:", best_cv_rmse_red, "params:", best_params_red)

# Train final OA-LSTM on reduced feature set
final_lstm_red = build_lstm(
    input_shape=(1, X_train_red_s.shape[1]),
    units=best_params_red["units"],
    dropout=best_params_red["dropout"],
    lr=best_params_red["lr"]
)
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=20, restore_best_weights=True)
final_lstm_red.fit(
    X_train_red_seq, y_train,
    validation_split=0.25,
    epochs=best_params_red["epochs"],
    batch_size=best_params_red["batch_size"],
    verbose=0,
    callbacks=[es]
)
pred_oa_red = final_lstm_red.predict(X_test_red_seq, verbose=0).ravel()
print("OA-LSTM (reduced features) — TEST:", evaluate(y_test, pred_oa_red))


Gen 1/5 — best CV RMSE: 29.8697 | best params: {'units': 24, 'dropout': 0.2, 'lr': 0.001, 'batch_size': 4, 'epochs': 120}
Gen 2/5 — best CV RMSE: 29.7564 | best params: {'units': 24, 'dropout': 0.2, 'lr': 0.001, 'batch_size': 4, 'epochs': 120}
Gen 3/5 — best CV RMSE: 28.9501 | best params: {'units': 32, 'dropout': 0.3, 'lr': 0.001, 'batch_size': 4, 'epochs': 120}
Gen 4/5 — best CV RMSE: 28.9501 | best params: {'units': 32, 'dropout': 0.3, 'lr': 0.001, 'batch_size': 4, 'epochs': 120}
Gen 5/5 — best CV RMSE: 28.9501 | best params: {'units': 32, 'dropout': 0.3, 'lr': 0.001, 'batch_size': 4, 'epochs': 120}
Best OA-LSTM (reduced) CV RMSE: 28.950068996301287 params: {'units': 32, 'dropout': 0.3, 'lr': 0.001, 'batch_size': 4, 'epochs': 120}
OA-LSTM (reduced features) — TEST: {'RMSE': 24.080993570279457, 'R2': -26.265985183350885, 'SD': 5.090042516678624}


In [21]:
def row(name, scores):
    return {"Model": name, **scores}

results = []
results.append(row("Random Forest", evaluate(y_test, pred_rf)))
results.append(row("SVM",           evaluate(y_test, pred_svm)))
results.append(row("DNN (fixed)",   evaluate(y_test, pred_dnn)))
results.append(row("RNN",           evaluate(y_test, pred_rnn)))
results.append(row("LSTM",          evaluate(y_test, pred_lstm)))
results.append(row("OA-LSTM full",  evaluate(y_test, pred_oa) if 'pred_oa' in locals() else {"RMSE": np.nan, "R2": np.nan, "SD": np.nan}))
results.append(row("OA-LSTM BOA",   evaluate(y_test, pred_oa_red)))

import pandas as pd
pd.DataFrame(results).sort_values("RMSE")

,Model,RMSE,R2,SD
0,Random Forest,3.265921,0.498485,3.239636
1,SVM,3.965551,0.260600,3.965551
2,DNN (fixed),14.708909,-9.172631,11.682091
3,RNN,21.494908,-20.724188,4.463840
6,OA-LSTM BOA,24.080994,-26.265985,5.090043
4,LSTM,24.888530,-28.125333,4.343886
5,OA-LSTM full,NaN,NaN,NaN
